In [1]:
!nvidia-smi

Mon Dec 30 19:53:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.01    Driver Version: 418.87.01    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    26W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!ls

PersonAttributes.ipynb	README.md  hvc_annotations.csv	hvc_data.zip  resized


In [4]:
import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

# from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, Callback


# from keras.applications import VGG16
from keras.layers import Conv2D, BatchNormalization, Activation, Dropout, Flatten, Dense, Input, MaxPooling2D, GlobalAveragePooling2D
from keras.regularizers import l2
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [6]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [7]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15, random_state = 42)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [9]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,...,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
10416,resized/10418.jpg,0,1,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0
3495,resized/3496.jpg,0,1,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
6950,resized/6951.jpg,1,0,0,0,1,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1
5035,resized/5036.jpg,1,0,1,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,1,0
4410,resized/4411.jpg,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1


In [10]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(train_df, batch_size=64, shuffle=False)

In [11]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'gender': 2,
 'image_quality': 3,
 'age': 5,
 'weight': 4,
 'bag': 3,
 'pose': 3,
 'footwear': 3,
 'emotion': 4}

In [12]:
weight_decay = 1e-4
inputs = Input(shape=(224,224,3))

def get_conv_block(num_filters, prev_layer, kernel_size=3, strides=1):
    return BatchNormalization()(Activation('relu')(Conv2D(num_filters,
       kernel_size=kernel_size,
       padding='same',
       strides=strides,
       kernel_regularizer=l2(weight_decay),
       use_bias=False)(prev_layer)))

num_filters = 32
block1 = get_conv_block(num_filters, inputs)
block1 = get_conv_block(num_filters, block1)
block1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(block1)

num_filters = 64
block2 = get_conv_block(num_filters, block1)
block2 = get_conv_block(num_filters, block2)
block2 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2))(block2)

num_filters = 128
block3 = get_conv_block(num_filters, block2)
block3 = get_conv_block(num_filters, block3)
block3 = MaxPooling2D(pool_size=(2, 2),strides=(2, 2))(block3)

num_filters = 256
block4 = get_conv_block(num_filters, block3)
block4 = get_conv_block(num_filters, block4)

#bottleneck
num_filters = 16
block5 = get_conv_block(num_filters, block4, kernel_size=1)

#GAP
block6 = GlobalAveragePooling2D()(block5)

neck = block6

#multi label output
def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(8, activation="relu")(neck)
    return neck

def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", build_tower(neck))
image_quality = build_head("image_quality", build_tower(neck))
age = build_head("age", build_tower(neck))
weight = build_head("weight", build_tower(neck))
bag = build_head("bag", build_tower(neck))
footwear = build_head("footwear", build_tower(neck))
emotion = build_head("emotion", build_tower(neck))
pose = build_head("pose", build_tower(neck))


model = Model(
    inputs=inputs, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

model.summary()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 224, 224, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 224, 224, 32) 128         activation_1[0][

In [22]:
from keras.utils import plot_model
plot_model(model)

In [23]:
def lr_schedule(epoch, lr=1e-3):
    if epoch > 30:
        lr *= 1e-1
    elif epoch > 60:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [lr_reducer, lr_scheduler]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])

Learning rate:  0.001


In [24]:
model_info = model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=8,
    epochs=100,
    verbose=1
)

Epoch 1/100
360/360 [==============================] - 46s 127ms/step - loss: 8.1890 - gender_output_loss: 0.6828 - image_quality_output_loss: 0.9915 - age_output_loss: 1.5033 - weight_output_loss: 1.0486 - bag_output_loss: 0.9515 - footwear_output_loss: 0.9968 - pose_output_loss: 0.9490 - emotion_output_loss: 0.9919 - gender_output_acc: 0.5678 - image_quality_output_acc: 0.5439 - age_output_acc: 0.3819 - weight_output_acc: 0.6313 - bag_output_acc: 0.5497 - footwear_output_acc: 0.5378 - pose_output_acc: 0.6109 - emotion_output_acc: 0.7113 - val_loss: 8.0307 - val_gender_output_loss: 0.6928 - val_image_quality_output_loss: 0.9521 - val_age_output_loss: 1.4503 - val_weight_output_loss: 1.0386 - val_bag_output_loss: 0.9292 - val_footwear_output_loss: 1.0498 - val_pose_output_loss: 0.9441 - val_emotion_output_loss: 0.9121 - val_gender_output_acc: 0.5404 - val_image_quality_output_acc: 0.5516 - val_age_output_acc: 0.4010 - val_weight_output_acc: 0.6338 - val_bag_output_acc: 0.5391 - val_foo

360/360 [==============================] - 41s 113ms/step - loss: 7.2852 - gender_output_loss: 0.5125 - image_quality_output_loss: 0.8956 - age_output_loss: 1.4018 - weight_output_loss: 0.9717 - bag_output_loss: 0.8602 - footwear_output_loss: 0.7994 - pose_output_loss: 0.8543 - emotion_output_loss: 0.8927 - gender_output_acc: 0.7474 - image_quality_output_acc: 0.5687 - age_output_acc: 0.4002 - weight_output_acc: 0.6332 - bag_output_acc: 0.6048 - footwear_output_acc: 0.6508 - pose_output_acc: 0.6247 - emotion_output_acc: 0.7128 - val_loss: 7.6753 - val_gender_output_loss: 0.7188 - val_image_quality_output_loss: 0.9238 - val_age_output_loss: 1.4132 - val_weight_output_loss: 0.9678 - val_bag_output_loss: 0.8962 - val_footwear_output_loss: 0.9185 - val_pose_output_loss: 0.8352 - val_emotion_output_loss: 0.8965 - val_gender_output_acc: 0.5692 - val_image_quality_output_acc: 0.5547 - val_age_output_acc: 0.4015 - val_weight_output_acc: 0.6338 - val_bag_output_acc: 0.5653 - val_footwear_output

Epoch 16/100
360/360 [==============================] - 41s 113ms/step - loss: 6.4164 - gender_output_loss: 0.2670 - image_quality_output_loss: 0.8595 - age_output_loss: 1.3454 - weight_output_loss: 0.9358 - bag_output_loss: 0.8182 - footwear_output_loss: 0.7138 - pose_output_loss: 0.3947 - emotion_output_loss: 0.8497 - gender_output_acc: 0.8898 - image_quality_output_acc: 0.5884 - age_output_acc: 0.4102 - weight_output_acc: 0.6384 - bag_output_acc: 0.6408 - footwear_output_acc: 0.6923 - pose_output_acc: 0.8483 - emotion_output_acc: 0.7130 - val_loss: 6.5376 - val_gender_output_loss: 0.2563 - val_image_quality_output_loss: 0.8442 - val_age_output_loss: 1.3359 - val_weight_output_loss: 0.9375 - val_bag_output_loss: 0.8125 - val_footwear_output_loss: 0.7051 - val_pose_output_loss: 0.5461 - val_emotion_output_loss: 0.8582 - val_gender_output_acc: 0.8944 - val_image_quality_output_acc: 0.5931 - val_age_output_acc: 0.4174 - val_weight_output_acc: 0.6385 - val_bag_output_acc: 0.6359 - val_fo

Epoch 24/100
360/360 [==============================] - 41s 113ms/step - loss: 5.3962 - gender_output_loss: 0.1183 - image_quality_output_loss: 0.7257 - age_output_loss: 1.1193 - weight_output_loss: 0.7912 - bag_output_loss: 0.7081 - footwear_output_loss: 0.5312 - pose_output_loss: 0.1743 - emotion_output_loss: 0.8085 - gender_output_acc: 0.9547 - image_quality_output_acc: 0.6526 - age_output_acc: 0.4943 - weight_output_acc: 0.6901 - bag_output_acc: 0.7097 - footwear_output_acc: 0.7687 - pose_output_acc: 0.9367 - emotion_output_acc: 0.7128 - val_loss: 7.7804 - val_gender_output_loss: 1.2794 - val_image_quality_output_loss: 0.6998 - val_age_output_loss: 1.1555 - val_weight_output_loss: 0.8786 - val_bag_output_loss: 1.0624 - val_footwear_output_loss: 1.0933 - val_pose_output_loss: 0.3470 - val_emotion_output_loss: 0.8318 - val_gender_output_acc: 0.6623 - val_image_quality_output_acc: 0.6735 - val_age_output_acc: 0.4863 - val_weight_output_acc: 0.6650 - val_bag_output_acc: 0.5990 - val_fo

360/360 [==============================] - 40s 111ms/step - loss: 4.5968 - gender_output_loss: 0.0859 - image_quality_output_loss: 0.4538 - age_output_loss: 0.9582 - weight_output_loss: 0.7208 - bag_output_loss: 0.4934 - footwear_output_loss: 0.4299 - pose_output_loss: 0.1189 - emotion_output_loss: 0.7656 - gender_output_acc: 0.9673 - image_quality_output_acc: 0.8036 - age_output_acc: 0.5766 - weight_output_acc: 0.7119 - bag_output_acc: 0.8132 - footwear_output_acc: 0.8034 - pose_output_acc: 0.9562 - emotion_output_acc: 0.7168 - val_loss: 4.8703 - val_gender_output_loss: 0.0805 - val_image_quality_output_loss: 0.7500 - val_age_output_loss: 0.9631 - val_weight_output_loss: 0.6817 - val_bag_output_loss: 0.4344 - val_footwear_output_loss: 0.4539 - val_pose_output_loss: 0.1263 - val_emotion_output_loss: 0.7998 - val_gender_output_acc: 0.9693 - val_image_quality_output_acc: 0.6479 - val_age_output_acc: 0.5761 - val_weight_output_acc: 0.7349 - val_bag_output_acc: 0.8400 - val_footwear_output

...

Epoch 62/100
360/360 [==============================] - 40s 112ms/step - loss: 3.4105 - gender_output_loss: 0.0616 - image_quality_output_loss: 0.1956 - age_output_loss: 0.6023 - weight_output_loss: 0.3593 - bag_output_loss: 0.3090 - footwear_output_loss: 0.3516 - pose_output_loss: 0.1300 - emotion_output_loss: 0.4385 - gender_output_acc: 0.9725 - image_quality_output_acc: 0.9257 - age_output_acc: 0.7514 - weight_output_acc: 0.8592 - bag_output_acc: 0.8821 - footwear_output_acc: 0.8385 - pose_output_acc: 0.9506 - emotion_output_acc: 0.8277 - val_loss: 5.3482 - val_gender_output_loss: 0.7329 - val_image_quality_output_loss: 0.3304 - val_age_output_loss: 1.4317 - val_weight_output_loss: 0.3120 - val_bag_output_loss: 0.5746 - val_footwear_output_loss: 0.4591 - val_pose_output_loss: 0.1454 - val_emotion_output_loss: 0.3963 - val_gender_output_acc: 0.7954 - val_image_quality_output_acc: 0.8719 - val_age_output_acc: 0.3949 - val_weight_output_acc: 0.8685 - val_bag_output_acc: 0.7716 - val_fo

Epoch 70/100
360/360 [==============================] - 40s 112ms/step - loss: 3.3761 - gender_output_loss: 0.0651 - image_quality_output_loss: 0.2010 - age_output_loss: 0.5731 - weight_output_loss: 0.3400 - bag_output_loss: 0.2863 - footwear_output_loss: 0.3288 - pose_output_loss: 0.1267 - emotion_output_loss: 0.4330 - gender_output_acc: 0.9699 - image_quality_output_acc: 0.9240 - age_output_acc: 0.7668 - weight_output_acc: 0.8648 - bag_output_acc: 0.8936 - footwear_output_acc: 0.8532 - pose_output_acc: 0.9514 - emotion_output_acc: 0.8273 - val_loss: 7.0431 - val_gender_output_loss: 0.5163 - val_image_quality_output_loss: 0.2775 - val_age_output_loss: 0.4135 - val_weight_output_loss: 0.2084 - val_bag_output_loss: 1.3407 - val_footwear_output_loss: 0.5595 - val_pose_output_loss: 1.0504 - val_emotion_output_loss: 1.6514 - val_gender_output_acc: 0.8517 - val_image_quality_output_acc: 0.8935 - val_age_output_acc: 0.8350 - val_weight_output_acc: 0.9225 - val_bag_output_acc: 0.6918 - val_fo

Epoch 78/100
360/360 [==============================] - 40s 112ms/step - loss: 3.3165 - gender_output_loss: 0.0582 - image_quality_output_loss: 0.1764 - age_output_loss: 0.5740 - weight_output_loss: 0.3358 - bag_output_loss: 0.2634 - footwear_output_loss: 0.2981 - pose_output_loss: 0.1206 - emotion_output_loss: 0.4204 - gender_output_acc: 0.9735 - image_quality_output_acc: 0.9339 - age_output_acc: 0.7627 - weight_output_acc: 0.8671 - bag_output_acc: 0.9029 - footwear_output_acc: 0.8772 - pose_output_acc: 0.9567 - emotion_output_acc: 0.8296 - val_loss: 5.7369 - val_gender_output_loss: 0.1634 - val_image_quality_output_loss: 0.1768 - val_age_output_loss: 2.0851 - val_weight_output_loss: 0.9221 - val_bag_output_loss: 0.1749 - val_footwear_output_loss: 0.3657 - val_pose_output_loss: 0.2565 - val_emotion_output_loss: 0.5197 - val_gender_output_acc: 0.9396 - val_image_quality_output_acc: 0.9296 - val_age_output_acc: 0.2616 - val_weight_output_acc: 0.6115 - val_bag_output_acc: 0.9383 - val_fo

Epoch 86/100
360/360 [==============================] - 41s 113ms/step - loss: 3.2043 - gender_output_loss: 0.0662 - image_quality_output_loss: 0.1651 - age_output_loss: 0.5538 - weight_output_loss: 0.3035 - bag_output_loss: 0.2281 - footwear_output_loss: 0.2615 - pose_output_loss: 0.1069 - emotion_output_loss: 0.4119 - gender_output_acc: 0.9710 - image_quality_output_acc: 0.9428 - age_output_acc: 0.7746 - weight_output_acc: 0.8805 - bag_output_acc: 0.9161 - footwear_output_acc: 0.8929 - pose_output_acc: 0.9584 - emotion_output_acc: 0.8307 - val_loss: 3.6172 - val_gender_output_loss: 0.0157 - val_image_quality_output_loss: 0.2326 - val_age_output_loss: 0.4036 - val_weight_output_loss: 0.6446 - val_bag_output_loss: 0.2601 - val_footwear_output_loss: 0.5255 - val_pose_output_loss: 0.0671 - val_emotion_output_loss: 0.3569 - val_gender_output_acc: 0.9945 - val_image_quality_output_acc: 0.9146 - val_age_output_acc: 0.8319 - val_weight_output_acc: 0.7639 - val_bag_output_acc: 0.9049 - val_fo

360/360 [==============================] - 40s 111ms/step - loss: 3.1745 - gender_output_loss: 0.0657 - image_quality_output_loss: 0.1564 - age_output_loss: 0.5534 - weight_output_loss: 0.2984 - bag_output_loss: 0.2128 - footwear_output_loss: 0.2398 - pose_output_loss: 0.1136 - emotion_output_loss: 0.3938 - gender_output_acc: 0.9699 - image_quality_output_acc: 0.9428 - age_output_acc: 0.7734 - weight_output_acc: 0.8827 - bag_output_acc: 0.9206 - footwear_output_acc: 0.9029 - pose_output_acc: 0.9570 - emotion_output_acc: 0.8359 - val_loss: 7.1220 - val_gender_output_loss: 2.1509 - val_image_quality_output_loss: 0.7754 - val_age_output_loss: 0.4361 - val_weight_output_loss: 0.2095 - val_bag_output_loss: 0.6589 - val_footwear_output_loss: 0.4039 - val_pose_output_loss: 0.8402 - val_emotion_output_loss: 0.5028 - val_gender_output_acc: 0.6572 - val_image_quality_output_acc: 0.7507 - val_age_output_acc: 0.8147 - val_weight_output_acc: 0.9101 - val_bag_output_acc: 0.7483 - val_footwear_output

In [25]:
# Score trained model.
results = model.evaluate_generator(valid_gen, verbose=1)
dict(zip(model.metrics_names, results))

180/180 [==============================] - 16s 86ms/step


{'loss': 15.497084771262275,
 'gender_output_loss': 0.30423638543321024,
 'image_quality_output_loss': 3.8313065648078917,
 'age_output_loss': 0.4593281818760766,
 'weight_output_loss': 0.4359119684331947,
 'bag_output_loss': 3.9197689983579846,
 'footwear_output_loss': 5.007244143221113,
 'pose_output_loss': 0.044429559168768014,
 'emotion_output_loss': 0.32259608548548485,
 'gender_output_acc': 0.8722222222222222,
 'image_quality_output_acc': 0.29184027777777777,
 'age_output_acc': 0.8037326388888889,
 'weight_output_acc': 0.834375,
 'bag_output_acc': 0.3662326388888889,
 'footwear_output_acc': 0.38359375,
 'pose_output_acc': 0.9858506944444444,
 'emotion_output_acc': 0.8495659722222222}

In [27]:
!aws s3 cp ./PersonAttributes.ipynb s3://dl-ci-test/final/PersonAttributes.ipynb

upload failed: ./PersonAttributes.ipynb to s3://dl-ci-test/final/PersonAttributes.ipynb An error occurred (NoSuchBucket) when calling the PutObject operation: The specified bucket does not exist
